# Défi IA

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import eli5
import shap
import xgboost as xgb
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import train_test_split 
#from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import seaborn as sns
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
#from sklearn.preprocessing import StandardScaler

In [6]:
bdd_all = pd.read_csv("pricing_requests.csv")
bdd = bdd_all[["hotel_id", "price", "city", "date", "language", "mobile", "request_number"]]
bdd_test = bdd[:30000]
bdd_test

,hotel_id,price,city,date,language,mobile,request_number
0,300,145,copenhagen,2,hungarian,0,1.0
1,855,311,copenhagen,2,hungarian,0,1.0
2,757,151,copenhagen,2,hungarian,0,1.0
3,805,202,copenhagen,2,hungarian,0,1.0
4,930,120,copenhagen,2,hungarian,0,1.0
...,...,...,...,...,...,...,...
29995,584,489,paris,10,austrian,1,1.0
29996,868,247,paris,10,austrian,1,1.0
29997,735,206,paris,10,austrian,1,1.0
29998,722,205,paris,10,austrian,1,1.0


### On split nos données en un échantillon d'apprentissage et un échantillon test

In [7]:
bdd_dummies = pd.get_dummies(data=bdd_test, columns = ['hotel_id','city', 'language', 'mobile'], drop_first=True)
data_train,data_test = train_test_split(bdd_dummies,test_size=0.25,random_state=1)
X_train = data_train.loc[:,data_train.columns != "price"].copy()
Y_train = data_train.loc[:,"price"].copy()
X_test = data_test.loc[:,data_train.columns != "price"].copy()
Y_test = data_test.loc[:,"price"].copy()

# # On convertit les variables catégorielles en indicatrices 
# X_train = pd.get_dummies(data=X_train, columns = ['hotel_id','city', 'language', 'mobile','avatar_id'], drop_first=True)
# X_test = pd.get_dummies(data=X_test, columns = ['hotel_id','city', 'language', 'mobile','avatar_id'], drop_first=True)

In [6]:
X_train

,date,request_number,hotel_id_1,hotel_id_2,hotel_id_3,hotel_id_4,hotel_id_5,hotel_id_6,hotel_id_7,hotel_id_8,...,language_luxembourgish,language_maltese,language_polish,language_portuguese,language_romanian,language_slovakian,language_slovene,language_spanish,language_swedish,mobile_1
26340,44,2.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3877,24,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
27081,8,2.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5440,24,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
22164,10,2.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17289,14,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5192,17,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12172,3,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
235,1,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### On entraine nos modèles et on regarde l'erreur RMSE entre Y_test et Y_pred 

In [8]:
#LinearRegression
lr = LinearRegression().fit(X_train, Y_train)
y_pred_lr = lr.predict(X_test)
RMSE_lr = math.sqrt(mean_squared_error(Y_test, y_pred_lr))
print(RMSE_lr)

21.587790357851908


In [5]:
##RandomForestRegressor
rf = RandomForestRegressor().fit(X_train, Y_train)
y_pred_rf = rf.predict(X_test)
RMSE_rf = math.sqrt(mean_squared_error(Y_test,y_pred_rf))
print(RMSE_rf)


9.421641665866943


In [9]:
##Multi-Layer Perceptron
mlp = MLPRegressor().fit(X_train, Y_train)
y_pred_mlp = mlp.predict(X_test)
RMSE_mlp = math.sqrt(mean_squared_error(Y_test, y_pred_mlp))
print(RMSE_mlp)

6.475635403179154


Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.


In [7]:
##GradientBoostRegressor
gbr = GradientBoostingRegressor().fit(X_train, Y_train)
y_pred_gbr = gbr.predict(X_test)
RMSE_gbr = math.sqrt(mean_squared_error(Y_test, y_pred_gbr))
print(RMSE_gbr)

62.195342673958486


In [8]:
##AdaBoostRegressor
abr = AdaBoostRegressor().fit(X_train,Y_train)
y_pred_abr = abr.predict(X_test)
RMSE_abr = math.sqrt(mean_squared_error(Y_test, y_pred_gbr))
print(RMSE_abr)

62.195342673958486


In [26]:
##XGBoostRegressor
xgbr = xgb.XGBRFRegressor(verbose=0)
xgbr_fit = xgbr.fit(X_train, Y_train)
y_pred_xgbr = xgbr_fit.predict(X_test)
RMSE_xgbr = math.sqrt(mean_squared_error(Y_test,y_pred_xgbr))

[12:02:57] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [27]:
print(RMSE_xgbr)

65.8526995133057


In [9]:
parameters_for_testing = {
    'colsample_bytree':[0.4,0.8],
    'gamma':[0,0.03,0.3],
    'min_child_weight':[1.5,6,10],
    'learning_rate':[0.1,0.07],
    'max_depth':[3,5],
    'n_estimators':[10000],
    'reg_alpha':[1e-5, 0.75],
    'reg_lambda':[1e-5, 0.45],
    'subsample':[0.6,0.95]
}

gsearch1 = GridSearchCV(estimator = xgbr, param_grid = parameters_for_testing, n_jobs=6, verbose=10,scoring='neg_mean_squared_error')
xgbr_gsearch = gsearch1.fit(X_train, Y_train)
y_pred_xgbr_gsearch = xgbr_gsearch.predict(X_test)
RMSE_xgbr_gsearch = math.sqrt(mean_squared_error(Y_test,y_pred_xgbr_gsearch))
print(RMSE_xgbr_gsearch)

Fitting 5 folds for each of 576 candidates, totalling 2880 fits


### On regarde l'importance de chaque variable dans chacun des modèles entraînés

In [28]:
#feature_names = ["hotel_id", "city", "date", "language", "mobile", "avatar_id", "request_number"]
feature_names = X_train.columns
print(bdd.columns)

Index(['hotel_id', 'price', 'city', 'date', 'language', 'mobile',
       'request_number'],
      dtype='object')


In [33]:
## Avec Permutation Importance 
features_importance_dict = {}
for model, name in zip([xgbr], ['xgboost regressor']):
  plt.figure()
  permutation_import = PermutationImportance(model).fit(X_test,Y_test)
  features_importance = {'Feature_name':feature_names, 'Importance':permutation_import.feature_importances_}  
  features_importance = pd.DataFrame(features_importance) # dataframe containing the features names and their importance
  features_importance = features_importance.sort_values(['Importance'], ascending = True) # sort the dataframe by feature importance
  features_importance_dict[name] = features_importance #add the dataframe to your dictionnary
  ax = sns.barplot(x="Importance", y="Feature_name", data=features_importance) #plot the model's features importance
  plt.title(name)

KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

In [15]:
## Avec la methode de Shapley
explainer = shap.KernelExplainer(xgbr_fit.predict, X_train)
shap_values = explainer.shap_values(X_test.iloc[0:5, :])
shap.summary_plot(shap_values, X_test.iloc[0:5,:])

Using 79803 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 